In [1]:
# # Install required libraries
# import os

# try:
#     import selenium
# except ImportError:
#     os.system("pip install selenium")

# try:
#     import webdriver_manager
# except ImportError:
#     os.system("pip install webdriver-manager")

# try:
#     import pandas as pd
# except ImportError:
#     os.system("pip install pandas")

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [2]:
# Initialize Chrome with options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Start maximized for better visibility
chrome_options.add_argument("--disable-infobars")  # Disable infobars
chrome_options.add_argument("--disable-extensions")  # Disable extensions for simplicity
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def scroll_to_bottom(driver, pause_time=2):
    """
    Scroll to the bottom of the page to load dynamic content.
    """
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

driver.get("https://www.expedia.com/Hotel-Search?adults=2&children=&d1=2025-05-01&d2=2025-05-07&destination=Cancun%2C%20Quintana%20Roo%2C%20Mexico&endDate=2025-05-07&latLong=21.161907%2C-86.851524&mapBounds=&regionId=179995&rooms=1&semdtl=&sort=RECOMMENDED&startDate=2025-05-01&theme=&useRewards=false&userIntent=&pwaDialog=")
time.sleep(6)  # Wait for the page to load completely

# Extract hotel data
listing_container = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-stid="property-listing-results"]'))
)
hotel_links = listing_container.find_elements(By.CSS_SELECTOR, 'a[data-stid="open-hotel-information"]')
print(f"Found {len(hotel_links)} hotel links.")


Found 250 hotel links.


In [3]:
import pandas as pd

# Initialize a list to store all hotel data
hotel_data = []
location = "Cancun, Mexico"

for index, link in enumerate(hotel_links):
    try:
        href = link.get_attribute('href')
        print(f"\nProcessing link {index + 1}")

        # Open the link in a new tab using JavaScript
        driver.execute_script("window.open(arguments[0]);", href)
        time.sleep(3)  # Wait for the new tab to open

        # Switch to the new tab
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(15)  # Wait for the new page to load

        hotel_details = {"Location": location}

        # Extract Hotel Name
        try:
            # First, try to locate the hotel name with the old structure
            try:
                hotel_name_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "h1.uitk-heading.uitk-heading-3"))
                )
                hotel_details['Hotel Name'] = hotel_name_element.text.strip()
            except Exception:
                # If the old structure fails, try the new structure
                hotel_name_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "h1.uitk-heading.uitk-heading-4"))
                )
                hotel_details['Hotel Name'] = hotel_name_element.text.strip()

            print(f"Extracted Hotel Name: {hotel_details['Hotel Name']}")
        except Exception as e:
            print(f"Error extracting hotel name: {e}")
            hotel_details['Hotel Name'] = None

        # Extract Stars
        try:
            stars_container = driver.find_element(By.CSS_SELECTOR, "div.uitk-rating.uitk-spacing.uitk-spacing-margin-blockstart-one")
            stars_count = len(stars_container.find_elements(By.TAG_NAME, "svg"))
            hotel_details['Stars'] = stars_count
        except Exception as e:
            print(f"Error extracting stars for hotel {index + 1}: {e}")
            hotel_details['Stars'] = None

        # Extract Rating
        try:
            rating_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "span.uitk-badge-base-large.uitk-badge-positive > span.uitk-badge-base-text")
                )
            )
            hotel_details['Rating'] = rating_element.text.strip()
        except Exception as e:
            print(f"Error extracting rating: {e}")
            hotel_details['Rating'] = None

        # Extract Review Keyword
        try:
            review_summary_container = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-stid='content-hotel-reviewsummary']"))
            )
            review_keyword_element = review_summary_container.find_element(
                By.CSS_SELECTOR, "div.uitk-text.uitk-type-500.uitk-type-medium.uitk-text-default-theme"
            )
            review_keyword = review_keyword_element.text.strip()
            hotel_details['Review Keyword'] = review_keyword
            print(f"Extracted Review Keyword: {review_keyword}")
        except Exception as e:
            print(f"Error extracting review keyword: {e}")
            hotel_details['Review Keyword'] = None

        # Extract Services
        try:
            services_list = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "ul.uitk-typelist[data-stid='sp-content-list']"))
            )

            services_lis = services_list.find_elements(By.CSS_SELECTOR, "li[data-stid^='sp-content-item-']")
            service_count = len(services_lis)
            print(f"Number of services: {service_count}")

            services = []
            for i in range(service_count):
                try:
                    service_span = driver.find_element(
                        By.CSS_SELECTOR,
                        f"ul.uitk-typelist[data-stid='sp-content-list'] li[data-stid='sp-content-item-{i}'] span.uitk-text.uitk-type-300.uitk-text-default-theme.uitk-layout-flex-item"
                    )
                    services.append(service_span.text.strip())
                except Exception as e:
                    print(f"Error extracting service {i}: {e}")
                    services.append(None)

            hotel_details['Services'] = services
        except Exception as e:
            print(f"Error extracting services: {e}")
            hotel_details['Services'] = []

        # Extract Property Offers
        try:
            property_offers_container = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-stid="section-room-list"]'))
            )

            property_offers = property_offers_container.find_elements(By.CSS_SELECTOR, 'div[data-stid^="property-offer-"]')

            offers = []
            for offer_index, offer in enumerate(property_offers, start=1):
                offer_details = {}

                try:
                    room_name_element = offer.find_element(By.CSS_SELECTOR, "h3.uitk-heading.uitk-heading-6")
                    offer_details['Room Name'] = room_name_element.text.strip()
                except Exception as e:
                    print(f"Error extracting room name for offer {offer_index}: {e}")
                    offer_details['Room Name'] = None

                try:
                    price_summary_div = offer.find_element(By.CSS_SELECTOR, 'div[data-stid="price-summary"]')
                    room_charges_element = price_summary_div.find_element(
                        By.CSS_SELECTOR,
                        "div.uitk-text.uitk-type-500.uitk-type-medium.uitk-text-emphasis-theme"
                    )
                    offer_details['Room Charges'] = room_charges_element.text.strip()
                except Exception as e:
                    print(f"Error extracting room charges for offer {offer_index}: {e}")
                    offer_details['Room Charges'] = None

                offers.append(offer_details)

            hotel_details['Property Offers'] = offers
        except Exception as e:
            print(f"Error extracting property offers: {e}")
            hotel_details['Property Offers'] = []

        # Add hotel details to list
        hotel_data.append(hotel_details)
        print(f"Extracted Hotel Data: {hotel_details}")

        # Close the new tab
        driver.close()

        # Switch back to the main tab
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)

    except Exception as e:
        print(f"Error processing link {index + 1}: {e}")
        continue

# Export hotel data to a CSV file
df = pd.DataFrame(hotel_data)

# Normalize property offers if necessary
if 'Property Offers' in df.columns:
    df['Property Offers'] = df['Property Offers'].apply(lambda x: ', '.join(str(offer) for offer in x) if isinstance(x, list) else x)

# Save to CSV
output_filename = f"hotel_data_{location.replace(',', '').replace(' ', '_')}.csv"
df.to_csv(output_filename, index=False)
print(f"Data exported to {output_filename}")


Processing link 1
Extracted Hotel Name: Nizuc Resort and Spa
Extracted Review Keyword: Wonderful
Number of services: 6
Extracted Hotel Data: {'Location': 'Cancun, Mexico', 'Hotel Name': 'Nizuc Resort and Spa', 'Stars': 5, 'Rating': '9.2', 'Review Keyword': 'Wonderful', 'Services': ['Restaurant', 'Spa', 'On private beach', 'Pool', 'Bar', 'Pet friendly'], 'Property Offers': [{'Room Name': 'Room, 1 King Bed, Garden View', 'Room Charges': '$417'}, {'Room Name': 'Garden View Double', 'Room Charges': '$455'}, {'Room Name': 'Deluxe Room, 1 King Bed, Ocean View', 'Room Charges': '$528'}, {'Room Name': 'Ocean View Deluxe Double', 'Room Charges': '$601'}, {'Room Name': 'Junior Suite (Garden)', 'Room Charges': '$674'}, {'Room Name': 'Ocean Suite', 'Room Charges': '$747'}, {'Room Name': 'Junior Suite, Ocean View', 'Room Charges': '$894'}, {'Room Name': 'Private Pool Villa', 'Room Charges': '$967'}, {'Room Name': 'Private Pool Pavillion Villa', 'Room Charges': '$1,040'}, {'Room Name': 'Penthouse',


Processing link 4
Extracted Hotel Name: Breathless Cancun Soul Resort & Spa - Adults Only - All Inclusive
Extracted Review Keyword: Excellent
Number of services: 6
Error extracting room charges for offer 16: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x003634A3+25059]
	(No symbol) [0x002ECEA4]
	(No symbol) [0x001CBEC3]
	(No symbol) [0x0020FD86]
	(No symbol) [0x0020FFCB]
	(No symbol) [0x00205B01]
	(No symbol) [0x00231F44]
	(No symbol) [0x00205A24]
	(No symbol) [0x00232194]
	(No symbol) [0x0024B51E]
	(No symbol) [0x00231C96]
	(No symbol) [0x00203FAC]
	(No symbol) [0x00204F3D]
	GetHandleVerifier [0x00655613+3113811]
	GetHandleVerifier [0x0066A2DA+3199002]
	GetHandleVerifier [0x00662AB2+3168242]
	


Processing link 7
Extracted Hotel Name: GR Solaris Cancun & Spa - All Inclusive
Extracted Review Keyword: Very good
Number of services: 6
Extracted Hotel Data: {'Location': 'Cancun, Mexico', 'Hotel Name': 'GR Solaris Cancun & Spa - All Inclusive', 'Stars': 4, 'Rating': '8.0', 'Review Keyword': 'Very good', 'Services': ['Pool', 'Spa', 'Gym', 'Bar', 'All inclusive', 'Room service'], 'Property Offers': [{'Room Name': 'Deluxe Room Balcony', 'Room Charges': '$295'}, {'Room Name': 'Deluxe Room, Balcony Ocean View', 'Room Charges': '$354'}, {'Room Name': 'Deluxe Room, Balcony Lagoon View', 'Room Charges': '$325'}, {'Room Name': 'Deluxe Room, Balcony Ocean Front', 'Room Charges': '$383'}, {'Room Name': 'Panoramic room, Balcony Double Ocean View', 'Room Charges': '$413'}, {'Room Name': 'Romantic Room, Balcony Ocean View', 'Room Charges': '$442'}, {'Room Name': 'Deluxe Room, Balcony 1 King Bed, Jetted Tub, Oceanfront', 'Room Charges': '$442'}]}

Processing link 8
Extracted Hotel Name: Hyatt Viv


Processing link 14
Extracted Hotel Name: Hotel Riu Palace Peninsula - All Inclusive
Extracted Review Keyword: Excellent
Number of services: 6
Extracted Hotel Data: {'Location': 'Cancun, Mexico', 'Hotel Name': 'Hotel Riu Palace Peninsula - All Inclusive', 'Stars': 4, 'Rating': '8.8', 'Review Keyword': 'Excellent', 'Services': ['Pool', 'Restaurant', 'All inclusive', 'Breakfast included', 'Bar', 'Hot Tub'], 'Property Offers': [{'Room Name': 'Junior Suite (U)', 'Room Charges': '$351'}, {'Room Name': 'Suite, Jetted Tub, Ocean View (U)', 'Room Charges': '$407'}, {'Room Name': 'Junior Suite, Ocean View (U)', 'Room Charges': '$373'}, {'Room Name': 'Junior Suite, Jetted Tub (Villa | U)', 'Room Charges': '$373'}, {'Room Name': 'Junior Suite, Ocean View (Superior, U)', 'Room Charges': '$390'}, {'Room Name': 'Junior Suite, Jetted Tub, Ocean View (Villa - U)', 'Room Charges': '$395'}, {'Room Name': 'Junior Suite, 2 Bedrooms (U)', 'Room Charges': '$702'}, {'Room Name': 'Junior Suite, 2 Bedrooms, Oc


Processing link 19
Extracted Hotel Name: Hilton Cancun Mar Caribe All-Inclusive Resort
Extracted Review Keyword: Excellent
Number of services: 6
Error extracting room charges for offer 21: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x003634A3+25059]
	(No symbol) [0x002ECEA4]
	(No symbol) [0x001CBEC3]
	(No symbol) [0x0020FD86]
	(No symbol) [0x0020FFCB]
	(No symbol) [0x00205B01]
	(No symbol) [0x00231F44]
	(No symbol) [0x00205A24]
	(No symbol) [0x00232194]
	(No symbol) [0x0024B51E]
	(No symbol) [0x00231C96]
	(No symbol) [0x00203FAC]
	(No symbol) [0x00204F3D]
	GetHandleVerifier [0x00655613+3113811]
	GetHandleVerifier [0x0066A2DA+3199002]
	GetHandleVerifier [0x00662AB2+3168242]
	GetHandleVerifier [

...


Processing link 234
Extracted Hotel Name: Hotel Riu Palace Costa Mujeres - All Inclusive
Extracted Review Keyword: Excellent
Number of services: 6
Extracted Hotel Data: {'Location': 'Cancun, Mexico', 'Hotel Name': 'Hotel Riu Palace Costa Mujeres - All Inclusive', 'Stars': 4, 'Rating': '8.6', 'Review Keyword': 'Excellent', 'Services': ['Pool', 'All inclusive', 'Breakfast included', 'Bar', 'Room service', 'Housekeeping'], 'Property Offers': [{'Room Name': 'Junior Suite, Garden View (U)', 'Room Charges': '$339'}, {'Room Name': 'Junior Suite, Partial Ocean View (U)', 'Room Charges': '$356'}, {'Room Name': 'Junior Suite, Oceanfront (U)', 'Room Charges': '$388'}, {'Room Name': 'Junior Suite (Swim-Up, U)', 'Room Charges': '$404'}, {'Room Name': 'Junior Suite, Ocean View (Large, U)', 'Room Charges': '$528'}]}

Processing link 235
Extracted Hotel Name: Waldorf Astoria Cancun
Extracted Review Keyword: Wonderful
Number of services: 6
Error extracting room charges for offer 14: Message: no such e

Extracted Review Keyword: Excellent
Number of services: 4
Error extracting property offers: Message: 
Stacktrace:
	GetHandleVerifier [0x003634A3+25059]
	(No symbol) [0x002ECEA4]
	(No symbol) [0x001CBEC3]
	(No symbol) [0x0020FD86]
	(No symbol) [0x0020FFCB]
	(No symbol) [0x0024D952]
	(No symbol) [0x00231F44]
	(No symbol) [0x0024B51E]
	(No symbol) [0x00231C96]
	(No symbol) [0x00203FAC]
	(No symbol) [0x00204F3D]
	GetHandleVerifier [0x00655613+3113811]
	GetHandleVerifier [0x0066A2DA+3199002]
	GetHandleVerifier [0x00662AB2+3168242]
	GetHandleVerifier [0x00403310+680016]
	(No symbol) [0x002F57ED]
	(No symbol) [0x002F2A98]
	(No symbol) [0x002F2C35]
	(No symbol) [0x002E5890]
	BaseThreadInitThunk [0x75E67BA9+25]
	RtlInitializeExceptionChain [0x77CBC0CB+107]
	RtlClearBits [0x77CBC04F+191]

Extracted Hotel Data: {'Location': 'Cancun, Mexico', 'Hotel Name': '4th floor condo with ocean view, balcony, WiFi, rooftop pool - walk to beach', 'Stars': None, 'Rating': '8.6', 'Review Keyword': 'Excellent', 


Processing link 242
Extracted Hotel Name: Grand Residences Riviera Cancún All Inclusive
Extracted Review Keyword: Exceptional
Number of services: 6
Extracted Hotel Data: {'Location': 'Cancun, Mexico', 'Hotel Name': 'Grand Residences Riviera Cancún All Inclusive', 'Stars': 5, 'Rating': '9.4', 'Review Keyword': 'Exceptional', 'Services': ['All inclusive', 'Pool', 'Free airport shuttle', 'Restaurant', 'Spa', 'Bar'], 'Property Offers': [{'Room Name': 'Junior Suite with Jacuzzi', 'Room Charges': '$394'}, {'Room Name': 'Junior Suite Beachfront', 'Room Charges': '$402'}, {'Room Name': 'Suite, 1 Bedroom, Private Pool', 'Room Charges': '$660'}, {'Room Name': 'One Bedroom Master Suite Beachfront', 'Room Charges': '$676'}]}

Processing link 243
Extracted Hotel Name: Classic 2BR Condo pool near beach
Error extracting stars for hotel 243: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.uitk-rating.uitk-spacing.uitk-spacing-margin-blockstart-one"}
  (Ses


Processing link 246
Extracted Hotel Name: Villa Palmeras
Extracted Review Keyword: Very good
Number of services: 6
Extracted Hotel Data: {'Location': 'Cancun, Mexico', 'Hotel Name': 'Villa Palmeras', 'Stars': 4, 'Rating': '8.4', 'Review Keyword': 'Very good', 'Services': ['Pool', 'Breakfast available', 'Bar', 'Housekeeping', 'Air conditioning', 'Parking included'], 'Property Offers': [{'Room Name': 'Standard Room', 'Room Charges': '$37'}, {'Room Name': 'Standard double room', 'Room Charges': '$37'}, {'Room Name': 'Superior Room', 'Room Charges': '$50'}, {'Room Name': 'Junior Suite', 'Room Charges': '$58'}, {'Room Name': 'Family Room', 'Room Charges': '$64'}]}

Processing link 247
Extracted Hotel Name: SOLE & MARE
Extracted Review Keyword: Good
Number of services: 2
Extracted Hotel Data: {'Location': 'Cancun, Mexico', 'Hotel Name': 'SOLE & MARE', 'Stars': 2, 'Rating': '9.0', 'Review Keyword': 'Good', 'Services': ['Pool', 'Free WiFi'], 'Property Offers': [{'Room Name': 'Family Studio', 

Error extracting property offers: Message: 
Stacktrace:
	GetHandleVerifier [0x003634A3+25059]
	(No symbol) [0x002ECEA4]
	(No symbol) [0x001CBEC3]
	(No symbol) [0x0020FD86]
	(No symbol) [0x0020FFCB]
	(No symbol) [0x0024D952]
	(No symbol) [0x00231F44]
	(No symbol) [0x0024B51E]
	(No symbol) [0x00231C96]
	(No symbol) [0x00203FAC]
	(No symbol) [0x00204F3D]
	GetHandleVerifier [0x00655613+3113811]
	GetHandleVerifier [0x0066A2DA+3199002]
	GetHandleVerifier [0x00662AB2+3168242]
	GetHandleVerifier [0x00403310+680016]
	(No symbol) [0x002F57ED]
	(No symbol) [0x002F2A98]
	(No symbol) [0x002F2C35]
	(No symbol) [0x002E5890]
	BaseThreadInitThunk [0x75E67BA9+25]
	RtlInitializeExceptionChain [0x77CBC0CB+107]
	RtlClearBits [0x77CBC04F+191]

Extracted Hotel Data: {'Location': 'Cancun, Mexico', 'Hotel Name': '2 Story- Oceanfront Penthouse on Cancun Beach, Steps From Sand and Nightlife!', 'Stars': None, 'Rating': '10', 'Review Keyword': 'Exceptional', 'Services': ['Parking available', 'Pool', 'Air condition